# Лекция №4. Объектно-ориентированное программирование

*П.Н. Советов, РТУ МИРЭА*

В Питоне каждое значение является объектом какого-то класса (типа). С помощью ООП можно создавать объекты пользовательских классов. Основы ООП вам уже знакомы на примерах языков C++ и Java, поэтому сконцентрируемся на особенностях реализации этой парадигмы программирования в Питоне. Эти особенности связаны с динамической природой Питона.

## Классы

Класс определяет пространство имен, в котором находятся функции и другие атрибуты класса. Новый класс создается с помощью ключевого слова `class`:

In [6]:
class MyClass:
    some_var = 10 # Переменная класса

    print('MyClass')
    
    def foo(): # Статический метод
        return MyClass.some_var + 1   
   
print(MyClass.some_var, MyClass.foo())

MyClass
10 11


Обращение к атрибутам классов, как и в случае с модулями, осуществляется через `.`.

Обратите внимание, что блок внутри `class` выполняется, как и любой другой блок. Поэтому и сработал `print` в примере выше, хотя, разумеется, практического смысла в его добавлении не было.

## Объекты

Объекты, то есть экземпляры класса, создаются с помощью вызова имени класса, как функции. 

In [7]:
class Point:
    pass

p1, p2 = Point(), Point()
p1.x, p1.y = 1, 2 # Динамически добавляем атрибуты объекта (лучше так не делать)
p2.x, p2.y = 3, 4
print((p1.x, p1.y), (p2.x, p2.y))

(1, 2) (3, 4)


В примере выше был неявно вызван конструктор по умолчанию. С использованием переопределения конструктора, который имеет специальное имя — `__init__`, инициализацию объекта можно упростить:

In [8]:
import math

class Point:
    def __init__(self, x, y): # Метод конструктора
        self.x = x
        self.y = y
    
    def magnitude(self):
        return math.sqrt(self.x**2 + self.y**2)

p = Point(1, 2)
print(p, p.x, p.y)

<__main__.Point object at 0x0000018E722F7E80> 1 2


Обратите внимание на параметр `self` в конструкторе. Роль этого параметра аналогична `this` в C++, то есть `self` содержит таблицу с атрибутами только что созданного объекта. В Питоне, как известно, не используется явное определение переменных, поэтому обращение к атрибутам и добавление новых атрибутов объекта необходимо осуществлять с использованием префикса `self.`. Помните, что `self` в списке аргументов метода должен всегда присутствовать первым.

С использованием предопределенных атрибутов можно динамически выяснять различную информацию по поводу конкретных объектов и классов:

In [9]:
print(p.__class__) # Класс объекта
print(p.__class__.__name__) # Имя класса объекта

<class '__main__.Point'>
Point


## Методы

Рассмотрим следующий пример:

In [10]:
m = p.magnitude
print(m, m())

<bound method Point.magnitude of <__main__.Point object at 0x0000018E722F7E80>> 2.23606797749979


Под «bound method» подразумевается ссылка на метод, уже связанный со своим объектом. Поэтому в примере выше оказался возможным вызов этого метода в `m` без указания объекта.

С помощью встроенной функции `getattr` можно получить доступ к атрибуту класса по имени атрибута в виде строки:

In [11]:
getattr(p, 'magnitude')()

2.23606797749979

Специальные типы методов можно определить с помощью декораторов (о них мы еще поговорим в одной из следующих лекций):

In [12]:
class MyClass:
    def __init__(self, x):
        self.x = x
    
    @staticmethod # Статический метод
    def my_static_method(x):
        return x
    
    @classmethod # Метод класса
    def my_constructor(cls, x):
        return cls(x)
    
    @property # Вычисляемое свойство
    def size(self):
        return self.x * 10

print(MyClass.my_static_method(42))
mc = MyClass.my_constructor(2)
print(mc.x, mc.size)

42
2 20


Выведем на экран атрибуты объекта класса `Point`:

In [13]:
print(dir(p)) # 

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'magnitude', 'x', 'y']


В Питоне имеется целый ряд специальных или «магических» методов в духе `__метод__`, имеющих предопределенное назначение. Из примера видно, что у объекта имеется ряд методов, которые мы не определяли. Можно, в частности, переопределить `__str__` (вывод строкового представления объекта) и `__repr__` (вывод строкового представления кода, создающего объект):

In [14]:
def __repr__(self):
    return f'Point({self.x}, {self.y})'

def __str__(self):
    return f'({self.x}, {self.y})'

Point.__repr__ = __repr__ # Динамически добавляем атрибуты класса (лучше так не делать)
Point.__str__ = __str__
    
p = Point(1, 2)
print(p)
p

(1, 2)


Point(1, 2)

In [15]:
import math

class Point:
    def __init__(self, x, y): # Метод конструктора
        self.x = x
        self.y = y
    
    def __len__(self): # Перегрузка len()
        return int(math.sqrt(self.x**2 + self.y**2)) # Попробуйте убрать int()

p = Point(x=1, y=2)
len(p)

2

Подробная информация о магических методах, которые можно использовать для перегрузки операций, приведена в [официальной документации](https://docs.python.org/3/reference/datamodel.html#special-method-names).

## Наследование

Подкласс наследует от базового класса все атрибуты последнего. Имеется возможность переопределить часть этих атрибутов в подклассе:

In [16]:
class Mortal:
    pass
    
class Human(Mortal):
    def __str__(self):
        return 'Human'

class Socrates(Human):
    def __str__(self):
        return 'Socrates'

print(Socrates())

Socrates


Наследуемые классы перечисляются в скобках через запятую. В наших прошлых примерах скобки после имени класса отсутствовали, но базовый класс все же неявно был определен и он назывался `object`:

In [17]:
print(dir(object))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


При обращении к некоторому атрибуту объекта поиск осуществляется по следующим правилам:
1. Проверяются атрибуты объекта.
1. Проверяются атрибуты класса объекта.
1. Проверяются атрибуты базовых классов.

С помощью вызова `super()` можно явно обратиться к атрибутам базового класса:

In [18]:
class Socrates(Human):
    def __str__(self):
        return super().__str__()

print(Socrates())

Human


Последовательность поиска атрибута для конкретного класса можно узнать с помощью `__mro__` (MRO — Method Resolution Order):

In [19]:
Socrates.__mro__

(__main__.Socrates, __main__.Human, __main__.Mortal, object)

Проверка на то, является ли объект экземляром заданного класса (`isinstance`), а также на то, является ли класс подклассом заданного класса (`issubclass`), демонстрируется ниже:

In [20]:
isinstance(p, Point), issubclass(Human, Mortal)

(True, True)

## Множественное наследование

В Питоне поддерживается множественное наследование, но нужно отметить, что его использование, во многих случаях, не решает, а создает дополнительные проблемы. По этой причине множественное наследование используется, в основном, для создания «примесей» (mixins), сходных с интерфейсами из таких языков, как Java.

Примеси представляют собой классы, не предназначенные для отдельного использования. В примесях определен набор методов, которые добавляют некоторую функциональность использующему их классу:

In [21]:
class LogMixin:
    def log(self):
        print('Обновляем log-файл.')
        
class ConvertMixin:
    def convert(self):
        print('Конвертируем данные.')
        
class DoSomething(LogMixin, ConvertMixin):
    pass

ds = DoSomething()
ds.log()
ds.convert()

Обновляем log-файл.
Конвертируем данные.


## Полиморфизм

В Питоне, как и в других динамических языках, используется динамическое связывание: мы можем обратиться к атрибуту некоторого объекта, вне зависимости от класса этого объекта. Достаточно лишь, чтобы атрибут с искомым именем можно был обнаружить в процессе описанных выше этапов поиска. Нет необходимости создавать и перегружать виртуальный метод, как это делается, например, в C++.

В Питоне используется принцип, называемый «утиной типизацией» — «если нечто выглядит, крякает и плавает, как утка, то это утка». В результате легко осуществляется работа с объектами различных классов, при условии что у этих объектов имеются нужные нам имена методов.

## Инкапсуляция


По умолчанию в Питоне не используется инкапсуляция данных. Все атрибуты класса являются публичными и при наследовании доступ к этим атрибутам не ограничивается.

Некоторую «защиту от несанкционированного доступа» можно обеспечить с помощью с помощью использования `__` в начале имени атрибута:

In [22]:
class Protected:
    def __init__(self):
        self.__secret_attribute = 42
        
p = Protected()
hasattr(p, '__secret_attribute') # Проверка на наличие атрибута

False

Продемонстрированную «защиту», тем не менее легко обойти, поскольку суть ее в автоматическом добавлении имени класса к имени атрибута:

In [23]:
print(vars(p)) # Вывести атрибуты объекта
p._Protected__secret_attribute += 1
print(vars(p))

{'_Protected__secret_attribute': 42}
{'_Protected__secret_attribute': 43}


Для проверки того, что разрабатываемый класс реализует необходимый интерфейс (то есть набор методов), можно использовать абстрактные классы, см. [официальную документацию](https://docs.python.org/3/library/abc.html).

## Шаблоны проектирования

Шаблоны проектирования представляют собой типовые конструкции, которые решают ту или иную частную задачу в рамках объектно-ориентированной разработки. Хотя знакомство с шаблонами проектирования и необходимо в рамках изучения ООП, но относится к ним желательно критически, не следует применять их бездумно.

Шаблоны проектирования иногда называют «отсутствующими возможностями языка программирования», подразумевая, что в языках высокоуровневых шаблоны не играют существенной роли. Так, Питер Норвиг проанализировал использование шаблонов проектирования и [обнаружил](http://norvig.com/design-patterns/design-patterns.pdf), что 16 из 23 популярных шаблонов не используются в таком динамическом языке, как Лисп, или же формулируются в гораздо более простом виде, чем в C++. Этот результат можно отнести и к Питону, поскольку он является  динамическим языком и, подобно Лиспу, поддерживает элементы функционального програмирования.

Если вы по каким-то причинам пропустили знакомство с шаблонами проектирования, то можно посоветовать прочесть небольшую книгу [Game Design Patterns](https://gameprogrammingpatterns.com/contents.html).

## Упаковка и распаковка аргументов

Реализовать функцию с произвольным количеством аргументов можно следующим образом:

In [24]:
def func(x, *args, **kwargs):
    print(x, args, kwargs)
    
func('x', 1, 2, 3, k1='a', k2='b')

x (1, 2, 3) {'k1': 'a', 'k2': 'b'}


В этом примере в аргумент с одной зведочкой попадает кортеж со всеми позиционными аргументами, а в аргумент с двумя звездочками — словарь с именованными аргументами.

При необходимости, полученные аргументы можно распаковать с помощью `*` (кортеж) и `**` (словарь):

In [25]:
def my_print(*args, **kwargs):
    print('print', *args, **kwargs)
    
my_print(1, 2, 3, sep=',')

print,1,2,3


С использованием операции `*` возможны и более изощренные решения:

In [26]:
first, *middle, last = range(0, 10)
print(first, middle, last)
first, second, *rest = range(0, 10)
print(first, second, rest)
*firsts, last = range(0, 10)
print(firsts, last)

0 [1, 2, 3, 4, 5, 6, 7, 8] 9
0 1 [2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8] 9
